# RMSA Experimental Notebook
## Análisis comparativo de algoritmos de RMSA en redes ópticas elásticas

Este notebook implementa First-Fit, Parcel-Fit y Sliding-Fit para comparar desempeño en topologías reales (Eurocore, NSFNet, UKNet, USNet).

### Instalación de dependencias

In [1]:
!pip3 install numpy pandas plotly networkx

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


### Importaciones necesarias

In [2]:
import numpy as np
import pandas as pd
import os
import sys
import plotly.express as px
import plotly.graph_objects as go
import time
import random

# Añadir RMSA_project al path
base = os.path.abspath(os.path.join(os.getcwd(), '..'))
proj = os.path.join(base, 'RMSA_project')
if proj not in sys.path:
    sys.path.insert(0, proj)

### Importación de funciones del loader

In [6]:
from src.loader import load_routes_from_dir
from src.algorithms.first_fit import find_first_fit_and_allocate
from src.algorithms.parcel_fit import allocate_parcel_fit
from src.algorithms.sliding_fit import allocate_sliding_fit

### Definición de métricas y funciones auxiliares

In [7]:
def fragmentation_metric(capacity):
    """
    Calcula el número promedio de segmentos libres (contiguos) por enlace.
    Esto da una medida de fragmentación: más segmentos = mayor fragmentación.
    """
    segments = []
    for row in capacity:
        free = (row == 0).astype(int)
        seg = 0
        inblock = False
        for v in free:
            if v == 1 and not inblock:
                seg += 1
                inblock = True
            elif v == 0:
                inblock = False
        segments.append(seg)
    return float(np.mean(segments)) if len(segments) > 0 else 0.0

def spectrum_efficiency(capacity):
    """
    Calcula la eficiencia del espectro (utilización / capacidad total).
    """
    if capacity.size == 0:
        return 0.0
    return float(np.sum(capacity)) / capacity.size

### Runner de experimentos RMSA

In [8]:
def run_rmsa_experiment(topology_name, routes_df, enlaces, rutas_usuarios, n_slots=80, n_demands=1000, seed=0):
    """
    Ejecuta los tres algoritmos RMSA sobre una topología y demandas aleatorias.
    Retorna un diccionario con métricas agregadas.
    """
    random.seed(seed)
    np.random.seed(seed)
    
    # Generar demandas aleatorias
    demand_choices = [1, 2, 3, 4, 6, 8, 10]
    demand_list = []
    for _ in range(n_demands):
        ridx = random.randrange(0, len(routes_df))
        size = random.choice(demand_choices)
        demand_list.append((ridx, size))
    
    results = {}
    
    # First-Fit
    capacity_ff = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_ff = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = find_first_fit_and_allocate(capacity_ff, route_row, size)
        if not ok:
            blocked_ff += 1
    time_ff = time.time() - start_time
    results['first_fit'] = {
        'blocked': blocked_ff,
        'blocking_prob': blocked_ff / len(demand_list),
        'utilization': spectrum_efficiency(capacity_ff),
        'fragmentation': fragmentation_metric(capacity_ff),
        'time_s': time_ff
    }
    
    # Parcel-Fit
    capacity_pf = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_pf = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = allocate_parcel_fit(capacity_pf, route_row, size, parcel_size=2)
        if not ok:
            blocked_pf += 1
    time_pf = time.time() - start_time
    results['parcel_fit'] = {
        'blocked': blocked_pf,
        'blocking_prob': blocked_pf / len(demand_list),
        'utilization': spectrum_efficiency(capacity_pf),
        'fragmentation': fragmentation_metric(capacity_pf),
        'time_s': time_pf
    }
    
    # Sliding-Fit
    capacity_sf = np.zeros((len(enlaces), n_slots), dtype=np.int8)
    blocked_sf = 0
    start_time = time.time()
    for (ridx, size) in demand_list:
        route_row = rutas_usuarios[ridx]
        _, ok = allocate_sliding_fit(capacity_sf, route_row, size)
        if not ok:
            blocked_sf += 1
    time_sf = time.time() - start_time
    results['sliding_fit'] = {
        'blocked': blocked_sf,
        'blocking_prob': blocked_sf / len(demand_list),
        'utilization': spectrum_efficiency(capacity_sf),
        'fragmentation': fragmentation_metric(capacity_sf),
        'time_s': time_sf
    }
    
    return results

### Carga de topologías de ejemplo (Eurocore, NSFNet)

In [13]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
topologies = ['Eurocore', 'NSFNet', 'UKNet', 'USNet']
loaded_topos = {}

for topo_name in topologies:
    try:
        ruta = os.path.join(base_dir, 'Rutas', topo_name)
        routes_df, enlaces, rutas_usuarios = load_routes_from_dir(ruta)
        loaded_topos[topo_name] = {
            'routes_df': routes_df,
            'enlaces': enlaces,
            'rutas_usuarios': rutas_usuarios
        }
        print(f"{topo_name}: {len(routes_df)} routes, {len(enlaces)} edges")
    except Exception as e:
        print(f"Could not load {topo_name}: {e}")

Eurocore: 110 routes, 46 edges
NSFNet: 182 routes, 44 edges
UKNet: 420 routes, 78 edges
USNet: 2070 routes, 152 edges


### Ejecución de experimentos en topologías cargadas

In [14]:
all_results = {}

for topo_name, topo_data in loaded_topos.items():
    print(f"\n=== Running experiments on {topo_name} ===")
    results = run_rmsa_experiment(
        topo_name,
        topo_data['routes_df'],
        topo_data['enlaces'],
        topo_data['rutas_usuarios'],
        n_slots=80,
        n_demands=500,  # reducido para desarrollo
        seed=0
    )
    all_results[topo_name] = results
    
    for alg_name, metrics in results.items():
        print(f"  {alg_name}:")
        print(f"    blocking_prob: {metrics['blocking_prob']:.6f}")
        print(f"    utilization: {metrics['utilization']:.6f}")
        print(f"    fragmentation: {metrics['fragmentation']:.2f}")
        print(f"    time: {metrics['time_s']:.3f}s")


=== Running experiments on Eurocore ===
  first_fit:
    blocking_prob: 0.302000
    utilization: 0.672826
    fragmentation: 2.98
    time: 0.146s
  parcel_fit:
    blocking_prob: 0.308000
    utilization: 0.685054
    fragmentation: 2.37
    time: 0.291s
  sliding_fit:
    blocking_prob: 0.292000
    utilization: 0.664130
    fragmentation: 3.57
    time: 0.282s

=== Running experiments on NSFNet ===
  first_fit:
    blocking_prob: 0.410000
    utilization: 0.734943
    fragmentation: 3.57
    time: 0.175s
  parcel_fit:
    blocking_prob: 0.414000
    utilization: 0.762216
    fragmentation: 3.32
    time: 0.335s
  sliding_fit:
    blocking_prob: 0.406000
    utilization: 0.720170
    fragmentation: 4.11
    time: 0.289s

=== Running experiments on UKNet ===
  first_fit:
    blocking_prob: 0.444000
    utilization: 0.480128
    fragmentation: 3.23
    time: 0.194s
  parcel_fit:
    blocking_prob: 0.458000
    utilization: 0.491827
    fragmentation: 3.40
    time: 0.334s
  sliding_f

### Visualización de resultados con plotly

In [15]:
# Preparar datos para graficar: blocking probability por topología y algoritmo
plot_data_blocking = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_blocking.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Blocking Probability': metrics['blocking_prob']
        })

df_blocking = pd.DataFrame(plot_data_blocking)
fig_blocking = px.bar(
    df_blocking,
    x='Topología',
    y='Blocking Probability',
    color='Algoritmo',
    barmode='group',
    title='Probabilidad de bloqueo por topología y algoritmo RMSA',
    labels={'Blocking Probability': 'Probabilidad de bloqueo'}
)
fig_blocking.show()

/Users/diego/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:1979: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  sf: grouped.get_group(s if len(s) > 1 else s[0])


In [16]:
# Utilización de espectro
plot_data_util = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_util.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Utilización': metrics['utilization']
        })

df_util = pd.DataFrame(plot_data_util)
fig_util = px.bar(
    df_util,
    x='Topología',
    y='Utilización',
    color='Algoritmo',
    barmode='group',
    title='Utilización de espectro por topología y algoritmo',
    labels={'Utilización': 'Eficiencia de espectro'}
)
fig_util.show()

/Users/diego/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [17]:
# Fragmentación
plot_data_frag = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        plot_data_frag.append({
            'Topología': topo_name,
            'Algoritmo': alg_name,
            'Fragmentación': metrics['fragmentation']
        })

df_frag = pd.DataFrame(plot_data_frag)
fig_frag = px.bar(
    df_frag,
    x='Topología',
    y='Fragmentación',
    color='Algoritmo',
    barmode='group',
    title='Fragmentación de espectro por topología y algoritmo',
    labels={'Fragmentación': 'Promedio de segmentos libres'}
)
fig_frag.show()

/Users/diego/Library/Python/3.9/lib/python/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



### Análisis de resultados

In [18]:
print("\n" + "="*60)
print("RESUMEN DE EXPERIMENTOS RMSA")
print("="*60)

for topo_name in sorted(all_results.keys()):
    print(f"\nTopología: {topo_name}")
    print("-" * 50)
    
    # Encontrar mejor algoritmo por métrica
    topo_results = all_results[topo_name]
    
    best_blocking = min(topo_results.items(), key=lambda x: x[1]['blocking_prob'])
    best_util = max(topo_results.items(), key=lambda x: x[1]['utilization'])
    best_frag = min(topo_results.items(), key=lambda x: x[1]['fragmentation'])
    
    print(f"  Menor bloqueo: {best_blocking[0]} ({best_blocking[1]['blocking_prob']:.6f})")
    print(f"  Mayor utilización: {best_util[0]} ({best_util[1]['utilization']:.6f})")
    print(f"  Menor fragmentación: {best_frag[0]} ({best_frag[1]['fragmentation']:.2f} segmentos)")

print("\n" + "="*60)


RESUMEN DE EXPERIMENTOS RMSA

Topología: Eurocore
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.292000)
  Mayor utilización: parcel_fit (0.685054)
  Menor fragmentación: parcel_fit (2.37 segmentos)

Topología: NSFNet
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.406000)
  Mayor utilización: parcel_fit (0.762216)
  Menor fragmentación: parcel_fit (3.32 segmentos)

Topología: UKNet
--------------------------------------------------
  Menor bloqueo: sliding_fit (0.442000)
  Mayor utilización: parcel_fit (0.491827)
  Menor fragmentación: first_fit (3.23 segmentos)

Topología: USNet
--------------------------------------------------
  Menor bloqueo: first_fit (0.340000)
  Mayor utilización: parcel_fit (0.467928)
  Menor fragmentación: first_fit (4.57 segmentos)



### Exportar resultados a CSV y JSON

In [ ]:
# Exportar como CSV
results_list = []
for topo_name, topo_results in all_results.items():
    for alg_name, metrics in topo_results.items():
        row = {'topology': topo_name, 'algorithm': alg_name}
        row.update(metrics)
        results_list.append(row)

df_results = pd.DataFrame(results_list)
results_dir = os.path.join(base_dir, 'results')
os.makedirs(results_dir, exist_ok=True)
csv_path = os.path.join(results_dir, 'rmsa_results.csv')
df_results.to_csv(csv_path, index=False)
print(f"Resultados exportados a {csv_path}")

# Exportar como JSON
json_path = os.path.join(results_dir, 'rmsa_results.json')
with open(json_path, 'w') as f:
    import json
    json.dump(all_results, f, indent=2)
print(f"Resultados exportados a {json_path}")

Resultados exportados a /Users/diego/Downloads/T2/RMSA_project/RMSA_project/results/rmsa_results.csv
Resultados exportados a /Users/diego/Downloads/T2/RMSA_project/RMSA_project/results/rmsa_results.json
